In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, GPT CBETA DC GUN example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

In [3]:
import yaml

# Make a proper input file. 

YAML="""
xopt: {output_path: null, verbose: true,
  algorithm: cnsga}
  
algorithm:
  name: cnsga
  options: {max_generations: 5, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
  selection: auto, verbose: true, population: null}

simulation:
  name: gpt_with_distgen
  evaluate: gpt.evaluate.evaluate_gpt_with_distgen

# Note that you can call another file in the top level group:
vocs: /Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/cbeta.dcgun.vocs.json

"""
config = yaml.safe_load(YAML)

In [4]:
# Create object
X = Xopt(config=config)
# Change some things to make it run faster
#X.vocs['constants']['lspch'] = True
#X.vocs['constants']['ipart'] = 1000
#X.vocs['constants']['zstop'] = 0.2

# Show config
X


            Xopt
________________________________           

Configured: True

Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: cnsga
  options: {max_generations: 5, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
    selection: auto, verbose: true, seed: null}
  max_generations: 5
  population_size: 8
  crossover_probability: 0.9
  mutation_probability: 1.0
  selection: auto
  verbose: true
  seed: null
simulation: {name: gpt_with_distgen, evaluate: gpt.evaluate.evaluate_gpt_with_distgen}
vocs:
  constants: {gun_voltage: 375}
  constraints:
    end_t_rms: [LESS_THAN, 0.0015]
  description: Test case
  name: CBETA DC GUN
  objectives: {end_x_coreemit_95percent: MINIMIZE, end_z_rms: MINIMIZE}
  simulation: gpt_with_distgen
  templates: {distgen_input_file: /Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json,
    gpt_input_file: /Users/colwyngulliford/Documents/Git

In [5]:
# Check vocs
X.vocs

{'constants': {'gun_voltage': 375},
 'constraints': {'end_t_rms': ['LESS_THAN', 0.0015]},
 'description': 'Test case',
 'name': 'CBETA DC GUN',
 'objectives': {'end_x_coreemit_95percent': 'MINIMIZE',
  'end_z_rms': 'MINIMIZE'},
 'simulation': 'gpt_with_distgen',
 'templates': {'distgen_input_file': '/Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json',
  'gpt_input_file': '/Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.gpt.in'},
 'variables': {'sol_1_current': [0, 5]},
 'linked_variables': None}

In [6]:
# Check random inputs
X.random_inputs()

{'sol_1_current': 4.964115112954467, 'gun_voltage': 375}

In [10]:
%%time
# Do a random evaluate to check that everything will run
x=X.random_evaluate()

CPU times: user 80 ms, sys: 24.5 ms, total: 105 ms
Wall time: 14.3 s


In [40]:
# These are the algorithm options
X.algorithm['options']

{'max_generations': 5,
 'population_size': 8,
 'crossover_probability': 0.9,
 'mutation_probability': 1.0,
 'selection': 'auto',
 'verbose': True,
 'seed': None}

In [46]:
# These are the options in the evaluate function
X.simulation['options']

{'archive_path': None,
 'merit_f': None,
 'distgen_input_file': '/Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json',
 'gpt_input_file': '/Users/colwyngulliford/Documents/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.gpt.in'}

# Run CNSGA using processes or threads

In [9]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [10]:
X.run_cnsga(executor)




 ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
  ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
░ ░               ░       ░        ░       ░  ░
░                                              


Continous Non-dominated Sorting Genetic Algorithm


Creating toolbox from vocs.
Created toolbox with 13 variables, 6 constraints, and 2 objectives.
    Using selection algorithm: nsga2
vocs:
{'constants': {'ipart': 1000, 'lspch': True, 'zstop': 0.2},
 'constraints': {'end_E_kinetic': ['GREATER_THAN', 90000000.0],
                 'end_deltaE_rms': ['LESS_THAN', 200000.0],
                 'end_higher_order_energy_spread': ['LESS_THAN', 5000.0],
                 'end_n_particle_loss': ['LESS_THAN', 1],
        

In [11]:
# Check for errors
X.population['errors']

[False, False, False, False, False, False, False, False]

# Write this configuration

In [12]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [13]:
!cat test.yaml

xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: cnsga
  options: {max_generations: 5, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
    selection: auto, verbose: true, seed: null}
  max_generations: 5
  population_size: 8
  crossover_probability: 0.9
  mutation_probability: 1.0
  selection: auto
  verbose: true
  seed: null
simulation: {name: astra_with_generator, evaluate: astra.evaluate.evaluate_astra_with_generator}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/kekgun/template/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/kekgun/template/dist004.in}
  variables:
    sig_x: [0.05, 1]
    lt: [0.005, 0.07]
    maxe(1): [20, 50]
    phi(1): [-30, 30]
    maxb(1): [0, 0.4]
    maxe(2): [0, 32]
    phi(2): [-180, 180]
    maxb(2): [0, 0.3]
    maxe(3): [0, 32]
    maxe(4): [0, 32]
    p

# Run with MPI

In [14]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.run_mpi test.yaml

  _   
                | |  
__  _____  _ __ | |_ 
\ \/ / _ \| '_ \| __|
 >  < (_) | |_) | |_ 
/_/\_\___/| .__/ \__|
          | |        
          |_|        

_________________________________
Parallel execution with 4 workers
Loading /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/test.yaml as YAML 




 ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
  ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
░ ░               ░       ░        ░       ░  ░
░                                              


Continous Non-dominated Sorting Genetic Algorithm


Creating toolbox from vocs.
Created toolbox with 13 variables, 6 constraints, and 2 objectives.
    Using selection algorithm: nsga2
vocs:
{'constants

# Dask 

In [15]:
from dask.distributed import Client
executor = Client()
executor

Client Scheduler: tcp://127.0.0.1:52287 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [16]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [17]:
# Increase the number of generations
X.algorithm['options']['max_generations'] = 10
X.run_cnsga(executor)




 ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
  ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
░ ░               ░       ░        ░       ░  ░
░                                              


Continous Non-dominated Sorting Genetic Algorithm


Creating toolbox from vocs.
Warning in cnsga_toolbox: Redefining creator.Individual
Created toolbox with 13 variables, 6 constraints, and 2 objectives.
    Using selection algorithm: nsga2
vocs:
{'constants': {'ipart': 1000, 'lspch': True, 'zstop': 0.2},
 'constraints': {'end_E_kinetic': ['GREATER_THAN', 90000000.0],
                 'end_deltaE_rms': ['LESS_THAN', 200000.0],
                 'end_higher_order_energy_spread': ['LESS_THAN', 5000.0],
          

# Cleanup

In [18]:
!rm -r dask-worker-space

In [19]:
!rm test.yaml